In [1]:
import numpy as np
from scipy.io import loadmat 
import sys
sys.path.append("/home2/ebrahim/neural_seq_decoder/scripts/")
from load_data import store_data_to_pandas
from matplotlib import pyplot as plt

In [2]:
fiftyWords = loadmat('/home2/ebrahim/neural_seq_decoder/tuningTasks/t12.2022.05.03_fiftyWordSet.mat')

blockNum = fiftyWords['blockNum']
fiftyWords['blockTypes'] = np.expand_dims(np.repeat('Words', blockNum.shape[0]),axis=-1)

words_pd = store_data_to_pandas(session_dict=fiftyWords, session_name='Words', stimuliKey='trialCues', zscore_bool=True)

In [29]:
fiftyWords['cueList']

array([[array(['DO NOTHING'], dtype='<U10'), array(['am'], dtype='<U2'),
        array(['are'], dtype='<U3'), array(['bad'], dtype='<U3'),
        array(['bring'], dtype='<U5'), array(['clean'], dtype='<U5'),
        array(['closer'], dtype='<U6'),
        array(['comfortable'], dtype='<U11'),
        array(['coming'], dtype='<U6'), array(['computer'], dtype='<U8'),
        array(['do'], dtype='<U2'), array(['faith'], dtype='<U5'),
        array(['family'], dtype='<U6'), array(['feel'], dtype='<U4'),
        array(['glasses'], dtype='<U7'), array(['going'], dtype='<U5'),
        array(['good'], dtype='<U4'), array(['goodbye'], dtype='<U7'),
        array(['have'], dtype='<U4'), array(['hello'], dtype='<U5'),
        array(['help'], dtype='<U4'), array(['here'], dtype='<U4'),
        array(['hope'], dtype='<U4'), array(['how'], dtype='<U3'),
        array(['hungry'], dtype='<U6'), array(['i'], dtype='<U1'),
        array(['is'], dtype='<U2'), array(['it'], dtype='<U2'),
        array(['

In [27]:
np.unique(fiftyWords['trialCues'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51],
      dtype=uint8)

In [4]:
words_pd_delay = words_pd.loc[words_pd.trialState==0]

In [5]:
words_pd_delay.head()

,blockNum,blockName,session,trialState,stimuli,trialNumber,bin_rel_go,audioEnvelope,pow-ip_6v-0,tx-ip_6v-0,...,pow-ip_44-251,tx-ip_44-251,pow-ip_44-252,tx-ip_44-252,pow-ip_44-253,tx-ip_44-253,pow-ip_44-254,tx-ip_44-254,pow-ip_44-255,tx-ip_44-255
0,2,Words,Words,0,28,0,-131,120.471709,-0.037352,-0.394279,...,0.319277,-0.391052,-0.147525,-0.179543,1.362292,1.291885,0.779324,1.567522,-0.271416,-0.493076
1,2,Words,Words,0,28,0,-130,105.149512,-0.734556,-0.394279,...,0.307419,-0.391052,0.177262,-0.179543,1.301280,-0.489134,-0.391820,-0.454313,0.200776,-0.493076
2,2,Words,Words,0,28,0,-129,71.570630,-0.576505,-0.394279,...,-0.099055,-0.391052,0.885181,-0.179543,0.498463,1.291885,-0.019679,-0.454313,-0.004540,-0.493076
3,2,Words,Words,0,28,0,-128,47.784127,0.677066,-0.394279,...,0.716807,-0.391052,1.024591,-0.179543,-0.242248,-0.489134,2.736732,-0.454313,1.013895,-0.493076
4,2,Words,Words,0,28,0,-127,38.418846,3.413317,4.562804,...,-0.156841,1.905989,1.584542,-0.179543,-0.392293,-0.489134,-0.031943,1.567522,-0.082634,0.665037


In [26]:
trialLengths = []
startTime = 5 # 100 ms 
fixedLength = 50 # 1000 ms
X = []
y = []
blockNums = []
silence_int = 1
stimulus_id_arr = []
sentence_lengths = []
for bn in np.unique(words_pd_delay.blockNum):
    
    words_pd_delay_bn = words_pd_delay.loc[words_pd_delay.blockNum==bn]
    
    print(words_pd_delay_bn.stimuli.unique().shape)
    
    for t in np.unique(words_pd_delay_bn.trialNumber):

        
        words_pd_delay_bn_t = words_pd_delay_bn.loc[words_pd_delay_bn.trialNumber==t]
        
        stimulus_id = words_pd_delay_bn_t.stimuli.iloc[0] - 1
            
        # Select columns with both 'tx-' and '6v' in their names
        tx_crossings = np.array(words_pd_delay_bn_t.filter(regex='tx-.*6v'))
        power_values = np.array(words_pd_delay_bn_t.filter(regex='pow-.*6v'))
        
        tx_crossings = tx_crossings[startTime:fixedLength]
        power_values = power_values[startTime:fixedLength]
        
        sentence_lengths.append(tx_crossings.shape[0])
        
        if tx_crossings.shape[0] < fixedLength-startTime:
            print("too short")
            
        neural_features_bn_t = np.hstack((tx_crossings, power_values))
        X.append(neural_features_bn_t)
        y.append(np.repeat(stimulus_id, fixedLength-startTime))
        blockNums.append(bn)


(51,)
(51,)
(51,)
(51,)
(51,)
(51,)
(51,)
(51,)
(51,)
(51,)


In [23]:
X_np = np.stack((X))
y_np = np.vstack((y))
blockNums_np = np.array(blockNums)
np.save('/data/LLMs/willet/delay_decode/moses_data/X', X_np)
np.save('/data/LLMs/willet/delay_decode/moses_data/y', y_np)
np.save('/data/LLMs/willet/delay_decode/moses_data/blockNums', blockNums_np)
np.save('/data/LLMs/willet/delay_decode/moses_data/sent_length', sentence_lengths)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51],
      dtype=uint8)

In [64]:

class NeuralDataDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # or torch.float32 if y is continuous

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
def getDatasetLoaders(train_ds, test_ds, batchSize):


    train_loader = DataLoader(
        train_ds,
        batch_size=batchSize,
        shuffle=True,
        num_workers=0,
    )
    
    test_loader = DataLoader(
        test_ds,
        batch_size=batchSize,
        shuffle=False,
        num_workers=0,
    )

    return train_loader, test_loader

In [66]:
from torch.utils.data import DataLoader

# Instantiate the dataset
dataset = NeuralDataDataset(X_np, y_np)

# Create the DataLoader
batch_size = 64  # You can set this to whatever value you need
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)


In [69]:
X, y = next(iter(data_loader))
print(X.shape, y.shape)

torch.Size([64, 151, 256]) torch.Size([64, 3])
